In [2]:
import tiktoken
import pandas as pd
from ast import literal_eval

In [5]:
df = pd.read_parquet('/dlabdata1/tsoares/linkrec-llms/data_modelling/benchmark_multilingual/test_data/en.parquet')
df.head(5)

,source_title,source_ID,target_title,context,section,mention_present,source_lead,first_version,second_version,direct_match,...,roberta_simple_multilingual,roberta_full_en,roberta_full_multilingual,roberta_dyn_mask_en,roberta_only_expansion_en,roberta_expansion_en,roberta_full_multilingual-all,entqa_rank,get_rank,roberta_dyn_mask_no_neg_en
0,2023%E2%80%9324_AFC_Cup_group_stage,74555776,Hettikamkanamge_Perera,Group A\nSource: Rules for classification: Gro...,Ranking of runner-up teams,False,AFC Cup group stage\nThe 2023–24 AFC Cup group...,1178732500,1178733621,True,...,1,1,1,3,1,1,1,3,2,1
1,Madonna_and_business,75009374,MCI_Group,"From 2017 to 2020, Madonna lived in Lisbon, Po...",Commercial impact,False,Aspect of Madonna's career\nAmerican singer-so...,1179712520,1179715108,False,...,119,21,1,66,30,76,55,107,17,14
2,2024_UEFA_Women%27s_Under-19_Championship_qual...,74509532,Bulgarian_Football_Union,Source: UEFARules for classification: Qualific...,Round 1,False,International football competition\nThe 2024 U...,1181645363,1181645710,False,...,12,11,2,39,1,6,1,41,32,19
3,Racism_in_Quebec,75111544,Bouchard-Taylor_Commission,"Historically, there has been a reluctance in Q...",History,True,Prejudice towards immigrants and minorities fr...,1181203098,1181204380,True,...,1,18,1,1,2,3,1,149,1,1
4,India%27s_Got_Talent_%28season_10%29,71081798,Ravi_Teja,The team with the lowest score goes to the eli...,Elimination Chart,False,Indian TV show\nThe tenth season of Indian tal...,1180115976,1180221202,True,...,1,5,3,1,3,3,4,10,11,1


In [7]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [20]:
for i in range(20):
    sample = df.sample(1)
    target_title = sample['target_title'].values[0]
    target_lead = sample['target_lead'].values[0]
    
    true_candidate = sample['context'].values[0]
    true_section = sample['section'].values[0]
    candidates = [true_candidate]
    sections = [true_section]
    
    for neg_candidate in literal_eval(sample['negative_contexts'].values[0]):
        candidates.append(neg_candidate['context'])
        sections.append(neg_candidate['section'])
    
    messages = [
        {
            'role': 'system',
            'content': 'You are RankGPT, an intelligent assistant that can rank passages based on their relevancy to the query.'
        },
        # The message below is possibly too long! Do we truncate the lead paragraph?
        {
            'role': 'user',
            'content': f'I will provide you with {{{{{len(candidates)}}}}} passages, each indicated by number identifie []. Rank them based on their relevance to query: {{{{{target_title} {target_lead}}}}}.'
        },
        {
            'role': 'assistant',
            'content': 'Okay, please provide the passages.'
        }
    ]

    for i, (candidate, section) in enumerate(zip(candidates, sections)):
        messages.append({
            'role': 'user',
            'content': f'[{i+1}] {{{{{section} {candidate}}}}}'
        })
        messages.append({
            'role': 'assistant',
            'content': f'Received passage [{i+1}].'
        })

    # This message can also get too long.
    messages.append({
        'role': 'user',
        'content': f'Search Query: {{{{{target_title} {target_lead}}}}}\nRank the {{{{{len(candidates)}}}}} passages above based on their relevance to the search query. The passages should be listed in descending order using identifiers, and the most relevant passages should be listed first, and the output format should be [] > [], e.g., [1] > [2]. Only response the ranking results, do not say any word or explain.'
    })

    tokens = encoding.encode(str(messages))
    print(f"Number of candidates: {len(candidates)}, Estimated number of tokens: {len(tokens)}")

Number of candidates: 3, Estimated number of tokens: 1986
Number of candidates: 14, Estimated number of tokens: 3244
Number of candidates: 658, Estimated number of tokens: 245429
Number of candidates: 18, Estimated number of tokens: 6137
Number of candidates: 64, Estimated number of tokens: 41311
Number of candidates: 16, Estimated number of tokens: 3570
Number of candidates: 48, Estimated number of tokens: 5504
Number of candidates: 147, Estimated number of tokens: 22210
Number of candidates: 47, Estimated number of tokens: 5907
Number of candidates: 122, Estimated number of tokens: 40219
Number of candidates: 77, Estimated number of tokens: 13877
Number of candidates: 39, Estimated number of tokens: 12489
Number of candidates: 7, Estimated number of tokens: 2277
Number of candidates: 2, Estimated number of tokens: 1033
Number of candidates: 62, Estimated number of tokens: 8911
Number of candidates: 26, Estimated number of tokens: 8616
Number of candidates: 554, Estimated number of to